<a href="https://colab.research.google.com/github/CocoPig1028/ML_Note/blob/main/EnsembleLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **랜덤 포레스트**

랜덤 포레스트의 경우 다음과 같이 학습
1. 1000개의 샘플이 들어있는 가방에서 100개를 다음과 같이 뽑는다.
2. 1개를 뽑고 다시 가방에 넣는다. 이 과정을 반복한다.

</br>이렇게 만들어진 샘플을 부트스트랩 샘플이라고 부름
</br>기본적으로 부트스트랩 샘플은 훈련 세트의 크기와 같게 만든다.

부트스트랩: 데이터 세트에서 중복을 허용하여 데이터를 샘플링하는 방식

</br>분류모델
</br>RandomForestClassifier: 기본적으로 전체 특성 개수의 제곱근만큼의 특성을 선택
</br>회귀모델
</br>RandomForestRegressor: 전체 특성을 사용

</br>분류: 샘플을 몇 개의 클래스 중 하나로 분류하는 문제
</br>회귀: 임의의 어떤 숫자를 예측하는 문제

In [2]:
# 사이킷런의 랜덤 포레스트는 기본적으로 100개의 결정 트리를 위와 방식으로 훈련
# 분류일 경우 각 트리의 클래스별 확률을 평균하여 가장 높은 확률을 가진 클래스를 예측으로 삼는다.
# 회귀일 경우 단순히 각 트리의 예측을 평균
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
# 훈련세트와 테스트 세트 분류

wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()
# 데이터와 타겟 설정

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [3]:
from sklearn.model_selection import cross_validate
# 교차검증
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs=-1, random_state=42)
# CPU 코어 모두 사용하여 랜덤포레스트 모델 생성

scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


랜덤 포레스트는 결정 트리의 앙상블이기 때문에 DecisionTreeClassifier가 제공하는 중요한 매개변수를 모두 제공
</br></br>
criterion, max_depth, max_features, min_samples_split, min_impurity_decrease, min_samples_leaf 등
</br>또한 결정 트리의 큰 장점 중 하나인 특성 중요도를 계산한다.
</br>랜덤 포레스트의 특성 중요도는 각 결정 트리의 특성 중요도를 취합한 것

In [4]:
rf.fit(train_input, train_target)
print(rf.feature_importances_)
# 알코올 도수, 당도, pH
# 1절 결정트리에서 만든 특성중요도 = [0.12345626, 0.86862934, 0.0079144]
# 당도의 중요도가 감소하고 알코올 도수와 pH의 특성 중요도가 상승
# 랜덤 포레스트가 특성의 일부를 랜덤하게 선택 -> 하나의 특성에 과도하게 집중하지 않음 -> 좀 더 많은 특성이 훈련에 기여할 기회를 획득 -> 과대적합을 줄이고 일반화 성능을 높임

[0.23167441 0.50039841 0.26792718]


In [5]:
# 부트스트랩 샘플에 포함되지 않고 남는 샘플 = OOB샘플
# 이 OOB샘플을 검증 세트로 사용가능
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
rf.fit(train_input, train_target)
print(rf.oob_score_)
# 교차 검증에서 얻은 점수와 비슷한 결과를 얻음
# OOB 점수와 교차 검증을 중복으로 사용할 필요는 없다.
# OOB 점수를 검증 점수처럼 사용하여 과대적합/과소적합을 조절하시면 된다.

0.8934000384837406


# **엑스트라 트리**

엑스트라 트리는 랜덤 포레스트와 매우 비슷하다.
</br>차이점으로는 엑스트라 트리의 경우 부트스트랩 샘플을 사용하지 않는다는 점이다.
</br>결정 트리를 만들 때 전체 훈련 세트를 사용한다는 뜻
</br>대신 노드를 분할할 때 가장 좋은 분할을 찾는 것이 아니라 무작위로 분할합니다.

In [6]:
# DecisionTreeClassifier의 splitter 매개변수를 'random'으로 지정
from sklearn.ensemble import ExtraTreesClassifier

et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))
# 엑스트라 트리가 무작위성이 좀 더 크기 때문에 랜덤 포레스트보다 더 많은 결정 트리를 훈련해야 한다.
# 하지만 랜덤하게 노드를 분할하기 때문에 빠른 계산 속도가 엑스트라 트리의 장점

0.9974503966084433 0.8887848893166506


In [7]:
et.fit(train_input, train_target)
print(et.feature_importances_)
# 여기도 당도의 중요도가 낮아졌다.
# 회귀할때에는 ExtraTreesRegressor

[0.20183568 0.52242907 0.27573525]


# **그레이디언트 부스팅**

그레이디언트 부스팅은 깊이가 얕은 결정 트리를 사용하여 이진 트리의 오차를 보완하는 방식으로 앙상블하는 방식
</br>사이킷런의 GradientBoostingClssifier는 기본적으로 깊이가 3인 결정 트리를 100개 사용
</br>깊이가 얕은 결정 트리를 사용 -> 과대적합에 강하고 높은 일반화 성능을 기대할 수 있음
</br></br>경사 하강법을 사용하여 틀리를 앙상블에 추가
</br>회귀에서는 평균 제곱 오차 함수를 사용
</br>
</br>경사 하강법: 함수의 기울기를 구하고 경사의 반대방향으로 이동시켜 극값에 이를때까지 반복
</br>4장에서 손실 함수를 산으로 정의하고 가장 낮은 곳을 찾아 내려오는 과정으로 설명, 이때 가장 낮은 곳을 찾아 내려오는 방법은 모델의 가중치와 절편을 조금씩 바꾸는 것이다.

In [8]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))
# 과대적합이 거의 되지 않는다.
# 그레이디언트 부스팅은 결정 트리의 개수를 늘려도 과대적합에 매우 적합
# 학습률을 증가시키고 트리의 개수를 늘리면 조금 더 성능이 향상될 수 있음

0.8881086892152563 0.8720430147331015


In [9]:
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))
gb.fit(train_input, train_target)
print(gb.feature_importances_)

0.9464595437171814 0.8780082549788999
[0.15872278 0.68010884 0.16116839]


# **히스토그램 기반 그레이디언트 부스팅**

정형 데이터를 다루는 머신러닝 알고리즘 중에 가장 인기가 높은 알고리즘
</br>입력 특성을 256개의 구간으로 나눈다.
</br>따라서 노드를 분할할 때 최적의 분할을 매우 빠르게 찾을 수 있다.
</br>히스토그램 기반 그레이디언트 부스팅은 256개의 구간 중에서 하나를 떼어 놓고 누락된 값을 위해서 사용
</br>누락된 특성이 있더라도 이를 따로 전처리 할 필요 없음


In [11]:
# HistGradientBoostingClassifier
# 일반적으로 기본 매개변수에서 안정적인 성능을 얻을 수 있음
# HistGradientBoostingClassifier에는 트리의 개수를 지정하는데 n_estimators대신에 부스팅 반복 회수를 지정하는 max_iter를 사용
# 이 클래스를 사용하기 위해서는 sklearn.experimental 패키지 아래에 있는 enable_hist_gradient_boosting 모듈을 임포트해야 함
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))
# 과대적합을 잘 억제하면서 그레이디언트 부스팅보다 조금 더 높은 성능을 제공

0.9321723946453317 0.8801241948619236


In [13]:
# permutation_importance()를 통해 히스토그램 기반 그레이디언트 부스팅의 특성 중요도를 계산
# 이 함수는 특성을 하나씩 랜덤하게 섞어서 모델의 성능이 변화하는지를 관찰하고 어떤 특성이 중요한지를 계산
# 훈련 세트 뿐만이 아닌 테스트 세트에도 적용할 수 있고 사이킷런에서 제공하는 추정기 모델에 모두 사용 가능
# 히스토그램 기반 그레이디언트 부스팅 모델 훈련 -> 훈련세트에서 특성 중요도 계산
# n_repeats 매개변수로 랜덤하게 섞을 횟수를 지정 -> 여기서는 10(기본값은 5)
from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances_mean)
# 반환하는 객체: 특성중요도, 평균, 표준편차 -> 여기서는 평균 뽑아옴

[0.08876275 0.23438522 0.08027708]


In [16]:
# 테스트 세트의 특성 중요도
result = permutation_importance(hgb, test_input, test_target, n_jobs=-1, random_state=42, n_repeats=10)
print(result.importances_mean)
# 그레이디언트 부스팅과 비슷하게 조금 더 당도에 집중하고 있음

[0.05969231 0.20238462 0.049     ]


In [17]:
hgb.score(test_input, test_target)
# 테스트 세트에서 약 87퍼센트의 정확도 획득
# 실전 투입 시 성능은 이보다 조금 낮을 것
# 앙상블 모델이 단일 결정 트리보다 좋은 결과를 얻을 수 있음

0.8723076923076923

In [18]:
# 히스토그램 기반 그레이디언트 부스팅의 회귀 버전인 HistGradientBoostingRegressor이 사이킷런에 구현
# 사이킷런 외의 대표적인 라이브러리는 XGBoost
# 사이킷런의 cross_validate() 함수와 함께 사용
# tree_method 매개변수를 hist로 지정하면 히스토그램 기반 그레이디언트 부스팅 사용가능
from xgboost import XGBClassifier
xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target, return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9555033709953124 0.8799326275264677


In [21]:
from lightgbm.sklearn import LGBMClassifier
# 마이크로소프트에서 만든 LightGBM도 또 다른 히스토그램 기반 그레이디언트 부스팅 라이브러리
# 빠르고 최신 기술을 많이 적용하고 있음
from lightgbm import LGBMClassifier
lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))
# 사이킷런의 히스토그램 기반 그레이디언트 부스팅이 LightGBM에서 영향을 많이 받음

0.935828414851749 0.8801251203079884


랜덤포레스트, 엑스트라트리, 그레이디언트 부스팅, 히스토그램 기반 그레이디언트 부스팅

4개의 앙상블을 모두 다루었음